In [2]:
import pandas as pd
import ast as ast
import numpy as np
from typing import *

from task_2_helpers import *
from collections import Counter


### Data Read

In [3]:
# df_business = pd.read_parquet('data/ATML2024_businesses.parquet')
# df_business = pd.read_csv('data/ATML2024_businesses.csv', header=0)

# df_Train_reviews = pd.read_parquet('data/ATML2024_reviews_train.parquet')
# df_Train_reviews = pd.read_csv('data/ATML2024_reviews_train.csv', header=0)

# # df_users = pd.read_parquet('data/ATML2024_users.parquet')
# df_users = pd.read_csv('data/ATML2024_users.csv', header=0, nrows=100)

### Cleanup business data columns

In [2]:
# bussiness_city = ['Phila','Philadelphia', 'Philadelphia', 'Philadelphia (Northeast Philly)', 'Philadelphia PA', 'Philadelphila', 'Philadephia', 'Philiadelphia', 
                #   'Philiidelphia', 'Philly', 'Southwest Philadelphia', 'West Philadelphia', 'philadelphia', 'PHILA', 'PHILADELPHIA']

In [3]:
# Convert dictionary columns to the correct types @@@@ Run only once
# df_business = df_business[df_business['city'].isin(bussiness_city)]
# df_business['attributes'] = df_business['attributes'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) or not pd.isnull(x)  else dict())
# df_business['hours'] = df_business['hours'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) or not pd.isnull(x)  else dict())

# # df_business['weekend'] = df_business['hours'].apply(lambda x: extract_weekend_value(x))
# df_business['hours_list'] = df_business['hours'].apply(lambda x: extract_hours_value(x))


In [4]:
# df_business['attributes_list'] = df_business['attributes'].apply(lambda x: sorted(extract_attributes_value(x)))
# df_business['categories_list'] = df_business['categories'].apply(lambda x: sorted(map(lambda a: a.strip().lower(), x.split(',')) ) if isinstance(x, str) else [x])

### Join Business Data With Ratings Data and Conduct Exploratory Data Analysis (EDA)

In [7]:
# df_business_data = df_business[['business_id', 'postal_code', 'attributes_list', 'hours_list', 'categories_list']]
# df_ratings_data = df_Train_reviews[['business_id', 'rating', 'useful', 'funny', 'cool', 'text']]

# df_data = pd.merge(df_business_data, df_ratings_data, left_on= 'business_id', right_on='business_id')
# df_data.head()

,business_id,postal_code,attributes_list,hours_list,categories_list,rating,useful,funny,cool,text
0,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,1,0,2,So I don't know how the other review was doubl...
1,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,0,I love this place.. I love their chunky fried...
2,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,1,I ordered so many cake from here for our famil...
3,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,0,0,0,Ordered a birthday cake here for the first tim...
4,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,1,0,0,This is my favorite bakery in Chinatown! It's ...


In [8]:
# df_data.to_parquet('data/ATML2024_Task2_PhiliBussRatings.parquet')
# print(df_data.shape)

### Explanatory Data Analysis

In [3]:
df_data = pd.read_parquet('data/ATML2024_Task2_PhiliBussRatings.parquet', engine='pyarrow')
print(df_data.shape)
print(df_data.columns)

(145048, 10)
Index(['business_id', 'postal_code', 'attributes_list', 'hours_list',
       'categories_list', 'rating', 'useful', 'funny', 'cool', 'text'],
      dtype='object')


In [10]:
# all_uniq_cat = set(np.concatenate(df_data.categories_list.values).ravel()) #df_data.categories_list.values.flatten()
# all_uniq_attr = set(np.concatenate(df_data.attributes_list.values).ravel()) #df_data.categories_list.values.flatten()

#### 2.0 Find out which one of the categories are related to restaurant services

Before deal with the reviews and treds in them, we need to identify which one of the businesses are actually in the restaurant and related area.

In [4]:
business_cat = np.concatenate(df_data.categories_list.values).ravel()
c = Counter(business_cat)

print(f'Total number of distinct categories of business: {len(c)}')

print(f'The most frequest categories : {c.most_common(10)}')

Total number of distinct categories of business: 1006
The most frequest categories : [('restaurants', 103030), ('food', 40393), ('nightlife', 35392), ('bars', 33605), ('american (new)', 23182), ('breakfast & brunch', 19437), ('american (traditional)', 17285), ('sandwiches', 16678), ('coffee & tea', 12653), ('italian', 12218)]


In [8]:
data = df_data[['business_id', 'categories_list']]
data_train = data.explode('categories_list')

In [10]:
data_train

,business_id,categories_list
0,wm9eoqjytVbC7dQcM4WSTM,bakeries
0,wm9eoqjytVbC7dQcM4WSTM,bubble tea
0,wm9eoqjytVbC7dQcM4WSTM,coffee & tea
0,wm9eoqjytVbC7dQcM4WSTM,food
0,wm9eoqjytVbC7dQcM4WSTM,restaurants
...,...,...
145045,QscEk0TPjLIlLgQzIN9TnW,restaurants
145046,QscEk0TPjLIlLgQzIN9TnW,mexican
145046,QscEk0TPjLIlLgQzIN9TnW,restaurants
145047,QscEk0TPjLIlLgQzIN9TnW,mexican


In [11]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import torch
from sklearn.cluster import KMeans

df_data = pd.read_parquet('data/ATML2024_Task2_PhiliBussRatings.parquet', engine='pyarrow')

data = df_data[['business_id', 'categories_list']]
data_train = data.explode('categories_list')



# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Get embeddings for each category
data_train['category_embedding'] = data_train['categories_list'].apply(lambda x: get_bert_embedding(x))

# Prepare embeddings for clustering
X = np.vstack(data_train['category_embedding'].values)

# Perform K-Means clustering
num_clusters = 10  # You can choose an appropriate number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
data_train['cluster'] = kmeans.fit_predict(X)

# Display the clusters
for cluster in range(num_clusters):
    print(f"Cluster {cluster}:")
    print(data_train[data_train['cluster'] == cluster]['category'].values)

ModuleNotFoundError: No module named 'transformers'

#### 2.1 Aspect Based Sentiment Analysis to discover the restaurants, service of restaurant, location of restaurant and food

In [11]:
df_data

,business_id,postal_code,attributes_list,hours_list,categories_list,rating,useful,funny,cool,text
0,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,1,0,2,So I don't know how the other review was doubl...
1,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,0,I love this place.. I love their chunky fried...
2,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,1,I ordered so many cake from here for our famil...
3,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,0,0,0,Ordered a birthday cake here for the first tim...
4,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,1,0,0,This is my favorite bakery in Chinatown! It's ...
...,...,...,...,...,...,...,...,...,...,...
145043,QaNNUNpPIvqk6LH1QjG4JL,19103,"[AcceptsInsurance, BikeParking, BusinessAccept...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[active life, doctors, fitness & instruction, ...",3,0,0,0,I've been to a few studios in the city and I c...
145044,QscEk0TPjLIlLgQzIN9TnW,19147,"[Alcohol, Ambience, BikeParking, BusinessAccep...","[[11.0, 22.0], [11.0, 22.0], [-1.0, -1.0], [11...","[mexican, restaurants]",5,2,1,1,First time trying this restaurant and I had a ...
145045,QscEk0TPjLIlLgQzIN9TnW,19147,"[Alcohol, Ambience, BikeParking, BusinessAccep...","[[11.0, 22.0], [11.0, 22.0], [-1.0, -1.0], [11...","[mexican, restaurants]",5,1,0,0,Amazing food and wonderful service! We got the...
145046,QscEk0TPjLIlLgQzIN9TnW,19147,"[Alcohol, Ambience, BikeParking, BusinessAccep...","[[11.0, 22.0], [11.0, 22.0], [-1.0, -1.0], [11...","[mexican, restaurants]",5,1,0,0,This hidden gem in south philly is why we need...


### 2.2 What is the best food of Philadelphia

Counter({'restaurants': 103030,
         'food': 40393,
         'nightlife': 35392,
         'bars': 33605,
         'american (new)': 23182,
         'breakfast & brunch': 19437,
         'american (traditional)': 17285,
         'sandwiches': 16678,
         'coffee & tea': 12653,
         'italian': 12218,
         'shopping': 10294,
         'pizza': 10233,
         'event planning & services': 9555,
         'chinese': 8650,
         'seafood': 8123,
         'beauty & spas': 8096,
         'specialty food': 7982,
         'desserts': 7609,
         'cocktail bars': 6981,
         'beer': 6435,
         'wine & spirits': 6435,
         'pubs': 6313,
         'burgers': 6286,
         'mexican': 6229,
         'arts & entertainment': 6200,
         'bakeries': 6051,
         'cafes': 6039,
         'asian fusion': 5980,
         'cheesesteaks': 5596,
         'japanese': 5590,
         'vegetarian': 5571,
         'salad': 5307,
         'lounges': 5083,
         'local flavor': 4

In [15]:
len(c.keys())

1006

In [21]:
# rating_1_cat = set(np.concatenate(df_data[df_data.rating == 1].categories_list.values).ravel())
# rating_2_cat = set(np.concatenate(df_data[df_data.rating == 2].categories_list.values).ravel())
# rating_3_cat = set(np.concatenate(df_data[df_data.rating == 3].categories_list.values).ravel())
# rating_4_cat = set(np.concatenate(df_data[df_data.rating == 4].categories_list.values).ravel())
# rating_5_cat = set(np.concatenate(df_data[df_data.rating == 5].categories_list.values).ravel())

In [ ]:
x = 2